# FastAPI + Uvicorn Demo

In [1]:
import json
import os
import subprocess
import webbrowser

import psutil
import requests

In [2]:
# remove database file if exists to start fresh
if os.path.isfile("app.db"):
    os.remove("app.db")

In [3]:
# readable output for JSON
def print_readable_json(x: dict, indent: int = 2):
    print(json.dumps(x, indent=indent))

In [4]:
# start and track a uvicorn subprocess
uvicorn_proc = subprocess.Popen(["python", "./app/uvicorn_host.py"])
print("uvicorn host process started with pid: {}".format(uvicorn_proc.pid))

uvicorn host process started with pid: 11192


In [5]:
print("uvicorn pid exists: {}".format(psutil.pid_exists(uvicorn_proc.pid)))

uvicorn pid exists: True


In [6]:
target = "http://localhost:5000"

In [7]:
# open the docs page for demonstrational purposes
webbrowser.open("{}/docs".format(target))

True

In [8]:
# test hello world
resp = requests.get(target)
print_readable_json(resp.json())

{
  "Hello": "World"
}


In [9]:
# make an items request
resp = requests.get("{}/items/1".format(target), params={"q": "first item description"})
print_readable_json(resp.json())

{
  "item_id": 1,
  "q": "first item description"
}


In [10]:
# send an improper request through
resp = requests.get(
    "{}/items/first".format(target), params={"q": "first item description"}
)
print_readable_json(resp.json())

{
  "detail": [
    {
      "loc": [
        "path",
        "item_id"
      ],
      "msg": "value is not a valid integer",
      "type": "type_error.integer"
    }
  ]
}


In [11]:
# get from the sklean dataset
resp = requests.get("{}/sklearn_wine".format(target))
sklearn_dataset = resp.json()
print_readable_json(sklearn_dataset[:1])

[
  {
    "uid": 0,
    "alcohol": 14.23,
    "malic_acid": 1.71,
    "ash": 2.43,
    "alcalinity_of_ash": 15.6,
    "magnesium": 127.0,
    "total_phenols": 2.8,
    "flavanoids": 3.06,
    "nonflavanoid_phenols": 0.28,
    "proanthocyanins": 2.29,
    "color_intensity": 5.64,
    "hue": 1.04,
    "od280_od315_of_diluted_wines": 3.92,
    "proline": 1065.0
  }
]


In [12]:
# post records to the database
resp = requests.post("{}/sqlite_wine".format(target), data=json.dumps(sklearn_dataset))
print_readable_json(resp.json())

{
  "processed": 178,
  "message": null
}


In [13]:
# read from the database
resp = requests.get("{}/sqlite_wine".format(target), params={"limit": "2"})
results = resp.json()
print_readable_json(results)

[
  {
    "uid": 0,
    "alcohol": 14.23,
    "malic_acid": 1.71,
    "ash": 2.43,
    "alcalinity_of_ash": 15.6,
    "magnesium": 127.0,
    "total_phenols": 2.8,
    "flavanoids": 3.06,
    "nonflavanoid_phenols": 0.28,
    "proanthocyanins": 2.29,
    "color_intensity": 5.64,
    "hue": 1.04,
    "od280_od315_of_diluted_wines": 3.92,
    "proline": 1065.0
  },
  {
    "uid": 1,
    "alcohol": 13.2,
    "malic_acid": 1.78,
    "ash": 2.14,
    "alcalinity_of_ash": 11.2,
    "magnesium": 100.0,
    "total_phenols": 2.65,
    "flavanoids": 2.76,
    "nonflavanoid_phenols": 0.26,
    "proanthocyanins": 1.28,
    "color_intensity": 4.38,
    "hue": 1.05,
    "od280_od315_of_diluted_wines": 3.4,
    "proline": 1050.0
  }
]


In [14]:
# kill the uvicorn subprocess
proc_killed = uvicorn_proc.kill()

In [15]:
print("uvicorn pid exists: {}".format(psutil.pid_exists(uvicorn_proc.pid)))

uvicorn pid exists: False
